In [1]:
from Game import *

Version:  0.2.41


In [2]:
from classy import *

Version  1.0.6
Version:  0.0.31


In [3]:
def take_picture(fname):
    pass

def get_square(arr,index,shape,locations=None):
    import json
    
    if locations is None:
        with open('locations.json') as json_file:
            locations = json.load(json_file)        
    
    try:
        location=locations[index]
    except IndexError:
        print("locations.json file probably corrupt.")
        raise 
        
    c,r=location
    c1=int(c-shape[1]/2)
    c2=int(c+shape[1]/2)
    r1=int(r-shape[0]/2)
    r2=int(r+shape[0]/2)

    c2=c2+(shape[1]-(c2-c1))
    r2=r2+(shape[0]-(r2-r1))

    square=arr[r1:r2,c1:c2,:]
    
    return square

In [4]:
def read_state_from_file(filename='gameboard.txt'):
    with open(filename) as fid:
        text=fid.read()

    text2=text.strip().split('\n')
    number_of_rows=len(text2)
    number_of_cols=len(text2[0].split())
    
    b=Board(number_of_rows,number_of_cols)
        
    board=[int(v) for v in text.split()]
    b.board=board
    return b

In [5]:
read_state_from_file(filename='gameboard.txt')

 1  1  1  1 
 0  0  0  0 
 0  0  0  0 
 2  2  2  2 

In [6]:

def read_state():
    from pylab import imread,imsave
    import os

    # train the classifier
    images=image.load_images('RotoPhotos2/squares/',delete_alpha=True)
    shape=images.data[0].shape[:2]
    data_train=data=image.images_to_vectors(images,verbose=True)  # train on all of them

    classifier=kNearestNeighbor()
    #classifier=NaiveBayes()
    classifier.fit(data_train.vectors,data_train.targets)


    # get the picture
    take_picture('board1.jpg')
    arr=imread('board1.jpg')


    # slice the picture into squares of the right size
    shape=data_train.shape[:2]
    squares=[get_square(arr,i,shape) for i in range(16)]
    test_images=image.array_to_image_struct(squares)

    # get predictions
    test_data=image.images_to_vectors(test_images)
    predictions=classifier.predict(test_data.vectors)
    
    if not os.path.exists('images'):
        os.mkdir('images')
    if not os.path.exists('images/predicted'):
        os.mkdir('images/predicted')
    for i,(square,prediction) in enumerate(zip(squares,predictions)):
        imsave('images/predicted/square %d predicted as %s.jpg' % (i,data_train.target_names[prediction]),square)

    
    # reconstruct the state from the predictions
    state=Board(4,4)
    for i in range(16):
        color=data_train.target_names[predictions[i]]
        if color=="Nothing":
            state[i]=0
        elif color=="White":
            state[i]=1
        elif color=="Black":
            state[i]=2
        else:
            raise ValueError("You can't get there from here.")

    print("Current state is:")
    print(state)

    x=input("""
    Hit return if this is correct, otherwise type a character 
    and the state will be read from current_board.txt.""")

    if x:
        state=read_state_from_file()

    print("Using")
    print(state)

    
    return state

In [7]:
state=read_state()

[Black]: 20 files found
[White]: 20 files found
[Nothing]: 40 files found
80 vectors of length 43200
Feature names: 'p0', 'p1', 'p2', 'p3', 'p4'  , ... ,  'p43195', 'p43196', 'p43197', 'p43198', 'p43199'  (43200 features)
Target values given.
Target names: 'Black', 'White', 'Nothing'
Mean:  [60.25   49.8875 90.625  ... 64.125  52.925  94.6625]
Median:  [60.  50.  90.  ... 62.5 52.5 93.5]
Stddev:  [ 7.23792097  7.25602121 10.94003542 ...  7.16131098  8.33332917
 11.51188923]
16 vectors of length 43200
Feature names: 'p0', 'p1', 'p2', 'p3', 'p4'  , ... ,  'p43195', 'p43196', 'p43197', 'p43198', 'p43199'  (43200 features)
Target values given.
Target names: 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None'
Mean:  [54.25   42.625  73.8125 ... 58.     47.5625 79.5   ]
Median:  [55.5 43.5 75.5 ... 55.5 43.  75.5]
Stddev:  [ 6.41774883  5.58877223  8.0017088  ...  9.29381515 14.99570251
 16.58312395]
Current state is:


    Hit return if this is correct, otherwise type a character 
    and the state will be read from current_board.txt. 


Using
 1  1  1  1 
 0  0  0  0 
 0  0  0  0 
 2  2  2  2 

